In [ ]:
import importlib.util
import subprocess

# Lista de paquetes que queremos instalar si no están presentes
packages = [
    "numpy",
    "pandas"
]

# Función para verificar e instalar paquetes
def install_if_missing(package):
    if importlib.util.find_spec(package) is None:
        print(f"Instalando {package}...")
        subprocess.check_call(["pip", "install", package])
    else:
        print(f"{package} ya está instalado.")

# Iterar sobre la lista de paquetes
for package in packages:
    install_if_missing(package)

Instalando openmeteo-requests...
Instalando requests-cache...
Instalando retry-requests...
numpy ya está instalado.
pandas ya está instalado.
Instalando ydata-profiling...
skforecast ya está instalado.


In [ ]:
#Importamos las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
from seaborn import heatmap
import matplotlib.dates as mdates
import streamlit as st
import pandas as pd
import io



In [ ]:

# Agregar opción para que el usuario suba un archivo
uploaded_file = st.file_uploader("Sube un archivo Excel", type=["xlsx"])

if uploaded_file is not None:
    excel_data = pd.ExcelFile(uploaded_file)

    # Mostrar las hojas disponibles y permitir al usuario elegir
    sheet_name = st.selectbox("Selecciona una hoja", excel_data.sheet_names)

    # Cargar la hoja seleccionada
    df = pd.read_excel(excel_data, sheet_name=sheet_name)

    st.write(f"Mostrando datos de la hoja: {sheet_name}")
    st.write(df)


# Cargar las hojas de datos para inspeccionar su contenido
monthly_data = df



# Buscar las primeras filas con datos válidos
monthly_data_clean = monthly_data.dropna(how='all').reset_index(drop=True)


# Definir los encabezados de las columnas
column_headers = [
    "Date",
    "Total_Operable_Units",
    "Net_Summer_Capacity_MW",
    "Net_Generation_MWh",
    "Share_of_Electricity_Percent",
    "Capacity_Factor_Percent"
]

# Extraer datos válidos de las tablas
monthly_data_extracted = monthly_data_clean.iloc[7:].reset_index(drop=True)
monthly_data_extracted.columns = column_headers

# Convertir las fechas a formato datetime donde sea posible
monthly_data_extracted['Date'] = pd.to_datetime(monthly_data_extracted['Date'], errors='coerce')

# Convertir las columnas numéricas
numeric_columns = column_headers[1:]
for col in numeric_columns:
    monthly_data_extracted[col] = pd.to_numeric(monthly_data_extracted[col], errors='coerce')


monthly_data_extracted.set_index('Date', inplace=True)



# Dropear las filas antes de diciembre de 1994 en el DataFrame original
monthly_data_extracted = monthly_data_extracted.loc['1994-12-01':]
#Variable por unidad operativa
monthly_data_per_unit = monthly_data_extracted.drop(columns=['Total_Operable_Units']).div(monthly_data_extracted['Total_Operable_Units'], axis=0)

# Convertir el DataFrame a un archivo Excel en memoria
output = io.BytesIO()
with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
    monthly_data_per_unit.to_excel(writer, sheet_name='Monthly Data Per Unit')

# Botón de descarga en Streamlit
st.download_button(
    label="Descargar archivo procesado",
    data=output.getvalue(),
    file_name="monthly_data_per_unit.xlsx",
    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
)